In [75]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [18]:
url = "https://www.digikala.com/ajax/product/comments/507136/"

querystring = {"page":"1","mode":"buyers"}

headers = {
    'User-Agent': "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0",
    'Accept': "*/*",
    'Accept-Language': "en-US,en;q=0.5",
    'Referer': "https://www.digikala.com/product/dkp-507136/%D9%84%D8%A7%D9%85%D9%BE-%D8%A7%D9%84-%D8%A7%DB%8C-%D8%AF%DB%8C-25-%D9%88%D8%A7%D8%AA-%D8%A8%D8%B1%D9%88%DA%A9%D8%B3-%D9%85%D8%AF%D9%84-1739-a95-%D9%BE%D8%A7%DB%8C%D9%87-e27?variant_id=956818",
    'Content-Type': "application/x-www-form-urlencoded; charset=UTF-8",
    'X-Requested-With': "XMLHttpRequest",
    'Connection': "keep-alive",
    'Cache-Control': "no-cache",
    'Postman-Token': "3dbd5dca-72df-4f38-862c-8f0c1edad5ee,2ea246f4-01fb-4652-ba3c-768f3641539f",
    'Host': "www.digikala.com",
    'Accept-Encoding': "gzip, deflate",
    'cache-control': "no-cache"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [81]:
def extract_comment_data(comment):
    # is_buyer
    try:
        if comment.find(class_="c-comments__buyer-badge").text == "خریدار":
            is_buyer = True
    except AttributeError as e:
        is_buyer = False
        return is_buyer, None, None, None
        
    # seller
    try:
        seller = comment.find(class_="o-text-blue").text
    except AttributeError as e:
        seller = comment.find(class_="btn-link-spoiler").text
        
    # date
    date = comment.find(
        lambda tag: tag.name == 'div' and tag.get('class') == ['cell']
        ).text.replace("در تاریخ", "").strip()
    
    # opinion
    try:
        opinion = comment.find(class_="c-message-light").text
    except AttributeError as e:
        opinion = None
        
    return is_buyer, seller, date, opinion

In [84]:
def parse_comment_page(response):
    soup = BeautifulSoup(response.text, 'html.parser')

    df = pd.DataFrame(columns=["product_id", "seller", "date", "opinion"])

    for comment in soup.find(class_="c-comments__list").findAll(class_="aside"):

        is_buyer, seller, date, opinion = extract_comment_data(comment)

        if is_buyer:
            df = df.append({
                'product_id': 507136,
                'seller': seller,
                'date': date,
                'opinion': opinion
            }, ignore_index=True)
        else:
            break
            
    return df

In [89]:
url = "https://www.digikala.com/ajax/product/comments/507136/"

headers = {
    'User-Agent': "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0",
    'Accept': "*/*",
    'Accept-Language': "en-US,en;q=0.5",
    'Referer': "https://www.digikala.com/product/dkp-507136/%D9%84%D8%A7%D9%85%D9%BE-%D8%A7%D9%84-%D8%A7%DB%8C-%D8%AF%DB%8C-25-%D9%88%D8%A7%D8%AA-%D8%A8%D8%B1%D9%88%DA%A9%D8%B3-%D9%85%D8%AF%D9%84-1739-a95-%D9%BE%D8%A7%DB%8C%D9%87-e27?variant_id=956818",
    'Content-Type': "application/x-www-form-urlencoded; charset=UTF-8",
    'X-Requested-With': "XMLHttpRequest",
    'Connection': "keep-alive",
    'Cache-Control': "no-cache",
    'Postman-Token': "3dbd5dca-72df-4f38-862c-8f0c1edad5ee,2ea246f4-01fb-4652-ba3c-768f3641539f",
    'Host': "www.digikala.com",
    'Accept-Encoding': "gzip, deflate",
    'cache-control': "no-cache"
    }

df = pd.DataFrame(columns=["product_id", "seller", "date", "opinion"])

for page in range(100):
    print("page: {}".format(page))
    print(df.shape)
    querystring = {"page":str(page),"mode":"buyers"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    df_new = parse_comment_page(response)
    df = pd.concat([df, df_new])

page: 0
(0, 4)
page: 1
(15, 4)
page: 2
(30, 4)
page: 3
(45, 4)
page: 4
(60, 4)
page: 5
(75, 4)
page: 6
(90, 4)
page: 7
(105, 4)
page: 8
(120, 4)
page: 9
(135, 4)
page: 10
(150, 4)
page: 11
(165, 4)
page: 12
(180, 4)
page: 13
(195, 4)
page: 14
(210, 4)
page: 15
(225, 4)
page: 16
(240, 4)
page: 17
(255, 4)
page: 18
(270, 4)
page: 19
(285, 4)
page: 20
(300, 4)
page: 21
(315, 4)
page: 22
(330, 4)
page: 23
(345, 4)
page: 24
(360, 4)
page: 25
(375, 4)
page: 26
(390, 4)
page: 27
(405, 4)
page: 28
(420, 4)
page: 29
(435, 4)
page: 30
(450, 4)
page: 31
(465, 4)
page: 32
(480, 4)
page: 33
(495, 4)
page: 34
(510, 4)
page: 35
(525, 4)
page: 36
(540, 4)
page: 37
(555, 4)
page: 38
(570, 4)
page: 39
(585, 4)
page: 40
(600, 4)
page: 41
(615, 4)
page: 42
(627, 4)
page: 43
(627, 4)
page: 44
(627, 4)
page: 45
(627, 4)
page: 46
(627, 4)
page: 47
(627, 4)


AttributeError: 'NoneType' object has no attribute 'findAll'

In [87]:
df.shape

(627, 4)

In [88]:
df

,product_id,seller,date,opinion
0,507136,کوثرالکتریک,۷ اردیبهشت ۱۳۹۸,خرید این محصول را توصیه می‌کنم
1,507136,دیجی‌کالا,۱۰ مرداد ۱۳۹۸,خرید این محصول را توصیه نمی‌کنم
2,507136,پریز نت,۹ اسفند ۱۳۹۷,خرید این محصول را توصیه می‌کنم
3,507136,دیجی‌کالا,۱۰ آبان ۱۳۹۷,خرید این محصول را توصیه می‌کنم
4,507136,دیجی‌کالا,۳۱ تیر ۱۳۹۸,خرید این محصول را توصیه می‌کنم
5,507136,دیجی‌کالا,۸ آبان ۱۳۹۸,خرید این محصول را توصیه می‌کنم
6,507136,دیجی‌کالا,۲۸ مرداد ۱۳۹۸,خرید این محصول را توصیه نمی‌کنم
7,507136,دیجی‌کالا,۲۰ شهریور ۱۳۹۸,خرید این محصول را توصیه می‌کنم
8,507136,دیجی‌کالا,۱۷ مرداد ۱۳۹۸,خرید این محصول را توصیه می‌کنم
9,507136,دیجی‌کالا,۲۵ شهریور ۱۳۹۸,در مورد خرید این محصول مطمئن نیستم
